In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# cut down to start at 2010
min_wage_file = "Raw_Data/Minimum_Wage_Data.csv"
min_wage_df = pd.read_csv(min_wage_file, encoding = "utf-8")
min_wage_df.head()

,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.1,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


In [4]:
# make column names lower case and simplify where able
pub_assistance_file = "Raw_Data/sub-est2016_all.csv"
pub_assistance_df = pd.read_csv(pub_assistance_file, encoding='utf-8')
pub_assistance_df.head()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334


In [7]:
pub_assistance_df.columns = ["geographic_summary_level","state_FIPS_code", "county_FIPS_code", "place_FIPS_code", "minor_civil_div_FIPS_code", "consolidated_city_FIPS_code", "primitive_geography_flag", "functional_status_code","city", "state", "census_pop_2010", 
                             "est_base_2010", "est_pop_2010", "est_pop_2011", "est_pop_2012", "est_pop_2013", "est_pop_2014", "est_pop_2015", "est_pop_2016"]

pub_assistance_df.head()

,geographic_summary_level,state_FIPS_code,county_FIPS_code,place_FIPS_code,minor_civil_div_FIPS_code,consolidated_city_FIPS_code,primitive_geography_flag,functional_status_code,city,state,census_pop_2010,est_base_2010,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334


In [10]:
SUMLEV_dict = {"geographic_summary_level": [40,50,61,71,157,162,170,172],
               "geographic_level": ["State", "County", "Minor Civil Division", "Minor Civil Division place part",
                                    "County place part", "Incorporated place", "Consolidated city",
                                    "Consolidated city -- place within consolidated city"]}

SUMLEV_df = pd.DataFrame(SUMLEV_dict)

In [13]:
new_pub_assist_df = pub_assistance_df.merge(SUMLEV_df, on='geographic_summary_level', how='left')
new_pub_assist_df.head()

,geographic_summary_level,state_FIPS_code,county_FIPS_code,place_FIPS_code,minor_civil_div_FIPS_code,consolidated_city_FIPS_code,primitive_geography_flag,functional_status_code,city,state,census_pop_2010,est_base_2010,est_pop_2010,est_pop_2011,est_pop_2012,est_pop_2013,est_pop_2014,est_pop_2015,est_pop_2016,geographic_level
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780131,4785492,4799918,4815960,4829479,4843214,4853875,4863300,State
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2688,2683,2685,2647,2631,2619,2616,2603,Incorporated place
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4522,4517,4495,4472,4447,4428,4395,4360,Incorporated place
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,756,754,753,748,748,747,740,738,Incorporated place
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,345,345,342,337,337,334,Incorporated place
